#Résumé de l'article : 

Le papier discute le sujet de l’augmentation de données pour achever des résultats plus élevé que les models de base (des models sans augmentation de données).

Le problème avec les jeux de données existants c’est qu’ils sont restreints à un nombre de domaine limité, ce qui pénalise les models.

Des solutions ont été déjà proposé dans l’état de l’art comme la continuité d’entrainement sur le domaine cible ou bien l’adaptation du corpus original par augmentation (ajout de bruit, cropping..), insertion aléatoire des mots. La méthode suivie dans l’article c’est celle de (Dai & Adel, 2020), le remplacement de mentions. Un des avantages de cette méthode par rapport aux autres, c’est sa simplicité et ne demande pas d’annotation.

**Idée de la méthode** : Remplacement d’une mention et ses mentions équivalentes par une autre entité de même type récupérée d’une liste prédéfinie au début.

Entité sur la quelle l’étude s’est concentré c’est PERSONNE.

L’entrainement s’est fait sur 5 configurations dont une sans augmentation de données.

Entraînement avec augmentation interne : où on utilise notre version “interne” des données en appliquant la méthode d’augmentation

Entraînement avec augmentation externe hors-domaine : où on utilise des données externes.

Entraînement avec augmentation externe intra-domaine : les données viennent de l’extérieur mais les entités nommées visées sont un peu different (dans l’article c’était nom personne dans le base, et les noms Fantasy dans l’externe)

Entraînement avec augmentation parfaite : on utilise ici des données externes mais contenant les entités nommées dans le jeu de données de base.
Les taux d’augmentation choisis sont : 5%, 10%, 50% et 100%

**Résultats et limites :**

3 métriques d’évaluation ont été observés, la précision, Rappel et le F1-score, un calcul des moyennes est fait sur les métriques mentionnés sur 25 entraînements.
On constate des gains pour le Rappel et le F1-score sur presque tout les types d’augmentation spécialement sur l’augmentation externe intera-domaine (fantasy) mais une précision qui ne s’améliore pas en incrémentant le taux d’augmentation.
Le résultat reste limité à un genre d’entités nommées (PERSONNE), et domain-specific.  De nouveaux domaines et d’autres types de combinaisons sont à explorés.


In [1]:
!git clone https://github.com/CompNet/ddaugNER.git

Cloning into 'ddaugNER'...
remote: Enumerating objects: 365, done.
remote: Counting objects: 100% (365/365), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 365 (delta 204), reused 302 (delta 147), pack-reused 0
Receiving objects: 100% (365/365), 1.11 MiB | 8.92 MiB/s, done.
Resolving deltas: 100% (204/204), done.


In [2]:
%cd ddaugNER/

/content/ddaugNER


In [3]:
!sudo apt-get update -y
!sudo apt-get install python3.9

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

#check python version
!python --version
#3.9.6

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,561 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:14 http://security.ub

In [4]:
!sudo apt-get install python3-pip
!sudo apt install python3.9-distutils
!python -m pip install --upgrade pip


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pkg-resources python3-secretstorage python3-setuptools python3-six
  python3-wheel python3-xdg
Suggested packages:
  python-crypto-doc python-cryptography-doc python3-cryptography-vectors
  gnome-keyring libkf5wallet-bin gir1.2-gnomekeyring-1.0
  python-secretstorage-doc python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pip python3-pkg-resources python3-secretstorage python3-setup

In [5]:
!pip install poetry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [6]:
!pip install -r requirements.txt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 442.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.5/895.5 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━

In [7]:

!python --version


Python 3.9.15


In [8]:
!pip install rich

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 11.0 MB/s eta 0:00:00
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached Pygments-2.13.0-py3-none-any.whl (1.1 MB)


In [9]:
!pip3 install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [10]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 890.2/890.2 MB 127.4 MB/s eta 0:00:01tcmalloc: large alloc 1112711168 bytes == 0x37b94000 @  0x7f7b6d46d615 0x60596b 0x543df1 0x543f6b 0x54403b 0x55c314 0x4ae709 0x5bcb62 0x5f4594 0x4ae709 0x5bcb62 0x5f4594 0x4ae709 0x5bcb62 0x5f4594 0x4ae709 0x5bcb62 0x5f4594 0x4ae709 0x5bcb62 0x5f4594 0x4ae709 0x5f44c4 0x4ae709 0x5bcb62 0x5f4604 0x4ae642 0x5bcb62 0x5f4594 0x4ae709 0x5bcb62
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.2 MB/s eta 0:00:00


In [11]:
!python setup_dekker_dataset.py

Cloning into 'Out-with-the-Old-and-in-with-the-Novel'...
remote: Enumerating objects: 788, done.
remote: Total 788 (delta 0), reused 0 (delta 0), pack-reused 788
Receiving objects: 100% (788/788), 18.88 MiB | 26.42 MiB/s, done.
Resolving deltas: 100% (165/165), done.
Book 'TheWheelOfTime' already has NER tags. Skipping...
Book 'StormFront' already has NER tags. Skipping...
Book 'TheWheelOfTime' already has NER tags. Skipping...
Book 'StormFront' already has NER tags. Skipping...


Traceback (most recent call last):
  File "/content/ddaugNER/train.py", line 2, in <module>
    from transformers import BertForTokenClassification
ModuleNotFoundError: No module named 'transformers'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 35.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=17590 sha256=3117a35d1b0fcb64b2888a6d0038b6ec9fac6a32f8e2b7acdd5c7cf6e4f1a0fe
  Stored in directory: /root/.cache/pip/wheels/9c/d6/00/1ccfd5a7466a94774e00022683d4b028836032dfb85007822b
Successfully built seqeval


In [15]:
!pip install --upgrade setuptools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 39.0.1
    Uninstalling setuptools-39.0.1:
      Successfully uninstalled setuptools-39.0.1


In [ ]:

!pip install seqeval

In [19]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 51.1 MB/s eta 0:00:00
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.0/770.0 kB 43.8 MB/s eta 0:00:00


In [20]:
!python train.py --help

usage: train.py
       [-h]
       [-en EPOCHS_NB]
       [-bz BATCH_SIZE]
       [-bm]
       [-mp MODEL_PATH]
       [-cw CUSTOM_WEIGHTS]
       [-cs CONTEXT_SIZE]
       [-cp CONLL_PERCENTAGE]
       [-koc [KEEP_ONLY_CLASSES ...]]
       [-tmp TEST_METRICS_PATH]
       [-das DATA_AUG_STRATEGIES]
       [-daf DATA_AUG_FREQUENCIES]
       [-dam DATA_AUG_METHOD]

optional arguments:
  -h, --help
    show this
    help
    message and
    exit
  -en EPOCHS_NB, --epochs-nb EPOCHS_NB
  -bz BATCH_SIZE, --batch-size BATCH_SIZE
  -bm, --batch-mode
  -mp MODEL_PATH, --model-path MODEL_PATH
  -cw CUSTOM_WEIGHTS, --custom-weights CUSTOM_WEIGHTS
    Custom
    class
    weights, as
    a json
    dictionay.
    Exemple :
    python
    train.py
    -cw
    '{"B-PER":
    2}'
    (default:
    None)
  -cs CONTEXT_SIZE, --context-size CONTEXT_SIZE
  -cp CONLL_PERCENTAGE, --conll-percentage CONLL_PERCENTAGE
    percentage
    of conll to
    use for
    training,
    between 0
    and 1 (who
    wo

In [ ]:
!python train.py\
       --epochs-nb 2\
       --batch-size 6\
       --context-size 1\
       --data-aug-strategies '{"PER": ["word_names"]}'\
       --data-aug-frequencies '{"PER": [0.1]}'\
       --model-path augmented_model.pth

In [ ]:
!python train.py\
       --epochs-nb 2\
       --batch-size 6\
       --context-size 1\
       --data-aug-strategies '{"PER": ["word_names"]}'\
       --data-aug-frequencies '{"PER": [1.0]}'\
       --model-path augmented_model.pth

In [21]:
!python train.py

running with config : 
{
    'epochs_nb': 5,
    'batch_size': 4,
    'batch_mode': False,
    'model_path': './ner_model',
    'custom_weights': None,
    'context_size': 0,
    'conll_percentage': 1.0,
    'keep_only_classes': None,
    'test_metrics_path': None,
    'data_aug_strategies': '{}',
    'data_aug_frequencies': '{}',
    'data_aug_method': 'standard'
}
Downloading: 100% 29.0/29.0 [00:00<00:00, 19.5kB/s]
Downloading: 100% 213k/213k [00:00<00:00, 859kB/s]
Downloading: 100% 436k/436k [00:00<00:00, 1.40MB/s]
Downloading: 100% 570/570 [00:00<00:00, 531kB/s]
Downloading: 100% 436M/436M [00:04<00:00, 94.0MB/s]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense

#10% of data augmentation

In [22]:
!python train.py\
       --epochs-nb 2\
       --batch-size 6\
       --context-size 1\
       --data-aug-strategies '{"PER": ["word_names"]}'\
       --data-aug-frequencies '{"PER": [0.1]}'\
       --model-path augmented_model.pth

running with config : 
{
    'epochs_nb': 2,
    'batch_size': 6,
    'batch_mode': False,
    'model_path': 'augmented_model.pth',
    'custom_weights': None,
    'context_size': 1,
    'conll_percentage': 1.0,
    'keep_only_classes': None,
    'test_metrics_path': None,
    'data_aug_strategies': '{"PER": ["word_names"]}',
    'data_aug_frequencies': '{"PER": [0.1]}',
    'data_aug_method': 'standard'
}
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForS

#100% of data augmentation

In [23]:
!python train.py\
       --epochs-nb 2\
       --batch-size 6\
       --context-size 1\
       --data-aug-strategies '{"PER": ["word_names"]}'\
       --data-aug-frequencies '{"PER": [1.0]}'\
       --model-path augmented_model.pth

running with config : 
{
    'epochs_nb': 2,
    'batch_size': 6,
    'batch_mode': False,
    'model_path': 'augmented_model.pth',
    'custom_weights': None,
    'context_size': 1,
    'conll_percentage': 1.0,
    'keep_only_classes': None,
    'test_metrics_path': None,
    'data_aug_strategies': '{"PER": ["word_names"]}',
    'data_aug_frequencies': '{"PER": [1.0]}',
    'data_aug_method': 'standard'
}
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForS

SyntaxError: ignored